# VAERS
---- 


# Disclaimer
**THIS IS NOT MEDICAL ADVICE**   
if you want medical advice please go see your doctor.  

please also read the disclaimer on the VAERS data on their website.  
[VAERS website](https://vaers.hhs.gov/data.html)



# get started
## 1. get the data

download the data from VAERS website  
[link to VAERS website](https://vaers.hhs.gov/data/datasets.html?)

this will give you AllVAERSDataCSVS.zip  

## 2. move and unzip
move the AllVAERSDataCSVS.zip to VAERS folder and unzip it  

## 3. check the files

you should have at least 6 files   
```
.../VAERS/AllVAERSDataCSVS/  
    2021VAERSVAX.csv  
    2021VAERSSYMPTOMS.csv  
    2021VAERSDATA.csv  
    2020VAERSVAX.csv  
    2020VAERSSYMPTOMS.csv  
    2020VAERSDATA.csv  
```

# THE CODE!

## setup/imports

In [72]:
# Import needed libraries
import pandas as pd
import numpy as np
import os
import sys
import re
import json
from collections import Counter

from IPython.display import display, HTML # needed for IPYNB
from matplotlib import pyplot as plt # we might used this one

In [73]:
# set pandas options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

## directory variables

DIR (String) : the current directory this file is in  
DATAPATH (String) : the location where the data files are stored

In [74]:

# DIR = os.path.dirname(os.path.realpath(__file__)) #doesn't work due to IPYNB file
DIR = '.' #this works in IPYNB
DATAPATH = os.path.join(DIR,'AllVAERSDataCSVS')

print('DATAPATH: ',DATAPATH)

DATAPATH:  .\AllVAERSDataCSVS


In [75]:
# not used ...but might use later

# def get_file_list(path):
#     """
#     returns a list of all the files in the given path/directory
#     """
#     return [ os.path.join(path,f) for f in os.listdir(path)]

# print(*get_file_list(DATAPATH),sep='\n')

## Saving and Loading JSON files

In [76]:
# LOADING AND SAVING JSON FILES

def load_json(path_to_file):
    """
    loads json from file
    """
    with open(path_to_file, 'r') as f:
        return json.load(f)

def save_json(path_to_file,data):
    """
    writes dict/json to file
    """
    with open(path_to_file, 'w') as f:
        json.dump(data, f,indent=4)

## test
# save_json(os.path.join(DATAPATH,'test.json'),{"HELLO":0})
# print(load_json(os.path.join(DATAPATH,'test.json')))

## processing files

the functions flow will process the csv files into one all_data.json file

In [77]:

def has_covid(text):
    """
    returns 1 or 0 if text has \'COVID\' in it
    """
    if re.search('COVID',text.upper()):
        return 1
    else:
        return 0
        
def compile_files(directory,files):
    """
    compiles/adds/unions multiple csv files together and returns a dataframe
    """
    df = pd.DataFrame()
    for i,f in enumerate(files):
        df0 = pd.read_csv(os.path.join(directory,f),encoding='cp1252',low_memory=False)
        df = pd.concat([df,df0])
    return df

def process_to_one_file():
    """
    processes VAERS data from 2020 and 2021, creates all_data.json, and returns a dataframe
    """
    print('process to one file\n\tthis might take a while...go get a drink☕🍷🍸🍹🍶🍺\n')
    df_data = compile_files(DATAPATH,['2020VAERSDATA.csv','2021VAERSDATA.csv'])
    df_vax = compile_files(DATAPATH,['2020VAERSVAX.csv','2021VAERSVAX.csv'])
    df_sym = compile_files(DATAPATH,['2020VAERSSYMPTOMS.csv','2021VAERSSYMPTOMS.csv'])

    # print("""
    # symptoms are contained in columns (up to 5 symptoms per event)
    # we must transform these symptoms into a single list for each event
    # """)
    print('dedup-ing Symptoms')
    vid = list(df_sym['VAERS_ID'].unique())

    symptom_columns = [
        'SYMPTOM1',
        'SYMPTOM2',
        'SYMPTOM3',
        'SYMPTOM4',
        'SYMPTOM5'
    ]

    idf_sym = []
    for index,v in enumerate(vid):
        if index%5000 == 0:
            print('{:.2f}'.format(index/len(vid)))
        temp = df_sym[df_sym['VAERS_ID'] == v]
        temp = temp.to_dict(orient='records')
        syms = []
        for t in temp:
            for sc in symptom_columns:
                if isinstance(t[sc],str):
                    syms.append(t[sc])
        idf_sym.append({'VAERS_ID':v,'SYMPTOMS':syms})
    df_sym = pd.DataFrame(idf_sym)

    print('merge data')
    df = pd.merge(df_data,df_vax,how='outer',on='VAERS_ID')
    df = df.drop_duplicates(ignore_index = True)
    df = pd.merge(df,df_sym,how='outer',on='VAERS_ID')

    df.reset_index()

    # creating a new column depending if this is the covid vaccine or not
    df['COVID_VAX'] = df['VAX_TYPE'].apply(has_covid)
    df = df[df['COVID_VAX'] == 1]
    # print(len(df))

    #save json file
    json_file = os.path.join(DATAPATH,'all_data.json')
    save_json(json_file,df.to_dict(orient='records'))
    print('saved: ',json_file)

    #save out csv file (not needed), but people might like a csv
    csv_file = os.path.join(DATAPATH,'all_data.csv') 
    df.to_csv(csv_file)
    print('saved: ',csv_file)

    return df


## get data
the below will get data from the files or from all_data.json.

df (DataFrame) :  contains all the datafrom VAERS files listed below
* .../VAERS/AllVAERSDataCSVS/  
    * 2021VAERSVAX.csv  
    * 2021VAERSSYMPTOMS.csv  
    * 2021VAERSDATA.csv  
    * 2020VAERSVAX.csv  
    * 2020VAERSSYMPTOMS.csv  
    * 2020VAERSDATA.csv 

In [78]:
def get_data():
    """
    gets the data and returns a dataframe
    """

    all_data = os.path.join(DATAPATH,'all_data.json')

    if os.path.isfile(all_data):
        print('loading all_data.json (15sec-30sec)')
        df = load_json(all_data)
        df = pd.DataFrame(df)
    else:
        print('processing the 2020-2021 files')
        print("""
        .../VAERS/AllVAERSDataCSVS/  
            2021VAERSVAX.csv  
            2021VAERSSYMPTOMS.csv  
            2021VAERSDATA.csv  
            2020VAERSVAX.csv  
            2020VAERSSYMPTOMS.csv  
            2020VAERSDATA.csv 
        """)
        df = process_to_one_file()

    return df

df = get_data()
print('\nloaded {:,} records/rows\n'.format(len(df)))
print('columns:\n',df.columns.to_list())
print('\n\ndf.head(10):\n')
display(df.head(10))

loading all_data.json (15sec-30sec)

loaded 448,897 records/rows

columns:
 ['VAERS_ID', 'RECVDATE', 'STATE', 'AGE_YRS', 'CAGE_YR', 'CAGE_MO', 'SEX', 'RPT_DATE', 'SYMPTOM_TEXT', 'DIED', 'DATEDIED', 'L_THREAT', 'ER_VISIT', 'HOSPITAL', 'HOSPDAYS', 'X_STAY', 'DISABLE', 'RECOVD', 'VAX_DATE', 'ONSET_DATE', 'NUMDAYS', 'LAB_DATA', 'V_ADMINBY', 'V_FUNDBY', 'OTHER_MEDS', 'CUR_ILL', 'HISTORY', 'PRIOR_VAX', 'SPLTTYPE', 'FORM_VERS', 'TODAYS_DATE', 'BIRTH_DEFECT', 'OFC_VISIT', 'ER_ED_VISIT', 'ALLERGIES', 'VAX_TYPE', 'VAX_MANU', 'VAX_LOT', 'VAX_DOSE_SERIES', 'VAX_ROUTE', 'VAX_SITE', 'VAX_NAME', 'SYMPTOMS', 'COVID_VAX']


df.head(10):



,VAERS_ID,RECVDATE,STATE,AGE_YRS,CAGE_YR,CAGE_MO,SEX,RPT_DATE,SYMPTOM_TEXT,DIED,DATEDIED,L_THREAT,ER_VISIT,HOSPITAL,HOSPDAYS,X_STAY,DISABLE,RECOVD,VAX_DATE,ONSET_DATE,NUMDAYS,LAB_DATA,V_ADMINBY,V_FUNDBY,OTHER_MEDS,CUR_ILL,HISTORY,PRIOR_VAX,SPLTTYPE,FORM_VERS,TODAYS_DATE,BIRTH_DEFECT,OFC_VISIT,ER_ED_VISIT,ALLERGIES,VAX_TYPE,VAX_MANU,VAX_LOT,VAX_DOSE_SERIES,VAX_ROUTE,VAX_SITE,VAX_NAME,SYMPTOMS,COVID_VAX
0,896636,11/14/2020,SC,47.0,47.0,NaN,F,NaN,"ARM SWELLING WITH PAIN, FEVER, FATIGUE, THEN ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,09/28/2020,10/02/2020,4.0,OCT 6 RESEARCH CENTER FOR ADVERSE SICK VISTIT\...,OTH,NaN,10mg Prozac daily,none,none,NaN,NaN,2,11/14/2020,NaN,Y,Y,tetracycline,COVID19,MODERNA,NaN,UNK,SYR,LA,COVID19 (COVID19 (MODERNA)),"[Arthralgia, Confusional state, Fatigue, Feeli...",1
1,902418,12/15/2020,NJ,56.0,56.0,NaN,F,NaN,Patient experienced mild numbness traveling fr...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,12/15/2020,12/15/2020,0.0,none,PVT,NaN,latex,none,none,NaN,NaN,2,12/15/2020,NaN,NaN,NaN,none,COVID19,PFIZER\BIONTECH,EH9899,1,IM,LA,COVID19 (COVID19 (PFIZER-BIONTECH)),"[Hypoaesthesia, Injection site hypoaesthesia]",1
2,902440,12/15/2020,AZ,35.0,35.0,NaN,F,NaN,C/O Headache,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,12/15/2020,12/15/2020,0.0,none,PVT,NaN,NaN,NaN,NaN,NaN,NaN,2,12/15/2020,NaN,NaN,NaN,NaN,COVID19,PFIZER\BIONTECH,EH 9899,1,SYR,LA,COVID19 (COVID19 (PFIZER-BIONTECH)),[Headache],1
3,902446,12/15/2020,WV,55.0,55.0,NaN,F,NaN,"felt warm, hot and face and ears were red and ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,12/15/2020,12/15/2020,0.0,She took Benadryl 50 mg po. No medical tests ...,OTH,NaN,"Novasc, Hydrochlorothiazide, synthroid, lisino...",none,"Hypertension, sleep apnea, hypothyroidism",NaN,NaN,2,12/15/2020,NaN,NaN,NaN,"Contrast Dye IV contrast, shellfish, strawberry",COVID19,PFIZER\BIONTECH,EH9899,1,IM,RA,COVID19 (COVID19 (PFIZER-BIONTECH)),"[Erythema, Feeling hot, Flushing]",1
4,902464,12/15/2020,LA,42.0,42.0,NaN,M,NaN,within 15 minutes progressive light-headedness...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,12/15/2020,12/15/2020,0.0,"Basic Chemistry, EKG, and vitals were all norm...",PVT,NaN,none,none,none,NaN,NaN,2,12/15/2020,NaN,NaN,Y,none,COVID19,PFIZER\BIONTECH,EH9899,UNK,IM,LA,COVID19 (COVID19 (PFIZER-BIONTECH)),"[Dizziness, Electrocardiogram normal, Hyperhid...",1
5,902465,12/15/2020,AR,60.0,60.0,NaN,F,NaN,Pt felt wave come over body @ 1218 starting in...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,12/15/2020,12/15/2020,0.0,BP.,PUB,NaN,"Lisinopril, HCTZ, lipitor, amyitrypline, lexap...","Bronchitis, finished prednisone on 12-13-20","hypertension, fibromyalgia",NaN,NaN,2,12/15/2020,NaN,NaN,NaN,Biaxin,COVID19,PFIZER\BIONTECH,EH9899,1,IM,RA,COVID19 (COVID19 (PFIZER-BIONTECH)),"[Dysgeusia, Oral pruritus, Paraesthesia, Parae...",1
6,902468,12/15/2020,NaN,59.0,59.0,NaN,M,NaN,"Within 1 minute, patient complained of symptom...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,12/15/2020,12/15/2020,0.0,Trransferred to ED,PVT,NaN,unknown,no,unknown,"vaso-vagal type symptoms, similar presentation...",NaN,2,12/15/2020,NaN,NaN,Y,unknown,COVID19,PFIZER\BIONTECH,EH9899,1,IM,LA,COVID19 (COVID19 (PFIZER-BIONTECH)),"[Chest discomfort, Chills, Defaecation urgency...",1
7,902479,12/15/2020,KS,46.0,46.0,NaN,F,NaN,rPfizer-BionNTech COVID-19 Vaccine EUA 5-7 mi...,NaN,NaN,NaN,NaN,Y,1.0,NaN,NaN,Y,12/14/2020,12/14/2020,0.0,Unknown but sent to SICU for monitoring,OTH,NaN,unknown,No,unknown,NaN,NaN,2,12/15/2020,NaN,NaN,NaN,No,COVID19,PFIZER\BIONTECH,NaN,1,IM,LA,COVID19 (COVID19 (PFIZER-BIONTECH)),"[Chest pain, Feeling abnormal, Flushing, Inten...",1
8,902490,12/15/2020,NM,37.0,37.0,NaN,F,NaN,"Headache, body ache",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,U,12/15/2020,12/15/2020,0.0,NaN,PVT,NaN,"Benzoyl peroxide 10% gel, Biofreeze topical ge...",None,"acne, astigmatism, myopia",NaN,NaN,2,12/15/2020,NaN,NaN,NaN,NKDA,COVID19,PFIZER\BIONTECH,EH9899,1,IM,LA,COVID19 (COVID19 (PFIZER-BIONTECH)),"[Headache, Pain]",1
9,902491,12/15/2020,NE,41.0,41.0,NaN,F,NaN,Within a few minutes of re

In [79]:
def print_row(items,column_lengths=[]):
    """
    comes in handly :)
    """
    row = ''
    for index,i in enumerate(items):
        try:
            cl = column_lengths[index]
        except IndexError:
            cl = 20
        row += str(i)[0:cl].ljust(cl)
    print(row)

## gets a list of symptoms

df_symptoms (DataFrame) : a list of all the symtoms and the counts of each 
all_symptoms (list): a list of all the symtoms and the counts of each 

**note:**   
symptoms might be medical jargon or plain english  
i.e. "RASH","ERYTHEMA", and "ITCHY RED SKIN"   
would be reported as different items (for now)

**note:**  
the counts/percentages below are of the symptoms.  
and one adverse reaction can have multiple symptoms.

In [103]:
def get_symptom_list(df,column='SYMPTOMS'):
    """
    returns a list of symptoms for the dataframe
    """
    s = df[column].to_list()
    l = []
    for i in s:
        try:
            for j in i:
                if str(j) == 'nan':
                    pass
                else:
                    l.append(str(j).upper())
        except:
            pass
    return l

all_symptoms =  get_symptom_list(df,'SYMPTOMS')
symptoms_count = len(all_symptoms)
all_symptoms = Counter(all_symptoms).most_common()

df_symptoms = pd.DataFrame(all_symptoms,columns=['SYMPTOM','COUNT'])
df_symptoms['PERCENT'] = (df_symptoms['COUNT']/symptoms_count)*100

file_name = os.path.join(DATAPATH,'symptoms.csv')
df_symptoms.to_csv(file_name)
print('saved: ',file_name)

topX = 100
print('below are the top {} symptoms'.format(topX))
display(df_symptoms.head(topX))


saved:  .\AllVAERSDataCSVS\symptoms.csv
below are the top 100 symptoms


,SYMPTOM,COUNT,PERCENT
0,HEADACHE,87781,4.479589
1,PYREXIA,71802,3.664158
2,FATIGUE,70952,3.620781
3,CHILLS,65995,3.367819
4,PAIN,64451,3.289026
5,DIZZINESS,52109,2.659196
6,NAUSEA,51148,2.610155
7,PAIN IN EXTREMITY,42680,2.178021
8,INJECTION SITE PAIN,32310,1.648825
9,ARTHRALGIA,27633,1.410151


## break down functions...
these functions will help me breakdown the data

### break_down_columns

In [81]:

def break_down_columns(idf,column):
    """
    shows what values there are for a given column (with counts and percent)
    """
    print('\nbreak down of {0}'.format(column))
    idf = pd.DataFrame(idf[column])
    idf = idf.fillna('nan')

    result = []

    l = list(idf[column].unique())
    for i in l:
        df0 = idf[idf[column]==i]
        result.append({'column':column,'value':i,'count':len(df0),'percent':(len(df0)/len(idf))*100})
    
    result = pd.DataFrame(result)
    display(result)
    return result

# test
# break_down_columns(df,'DIED')



### break_down_buckets

In [82]:


def break_down_buckets(idf,column,buckets,message='',nan_value=-1):
    """
    breaks a column down into buckets/bins
    """
    idf = idf.fillna(nan_value)

    print('',message,'\ncolumn: ',column, '\nbuckets: ', buckets)
    
    idf = pd.DataFrame(idf[column])
    idf['bucket'] = pd.cut(idf[column], bins=buckets)
    idf = idf.groupby(by='bucket').count()
    idf['percent'] = (idf[column]/idf[column].sum())*100
    idf['percent'] = idf['percent'].round(2)
    display(idf)

    return idf

#test
# break_down_buckets(df,'AGE_YRS',[-1,0,15,25,35,45,55,65,75,85,500])


## Additional Numbers

these numbers are **not** part of the vaers data,  
however they are important to analyzing the data

vaxx (int) : the number of vaccinated (1 or more shots) US citizens according to a quick google search (on 8/3/2021)

full_vaxx (int) : the number of vaccinated (2 or more shots) US citizens according to a quick google search (on 8/3/2021)


In [83]:
vaxx = 191*10**6
full_vaxx = 165*10**6

In [84]:
#only death vaers
df_death = df[df['DIED']=='Y']

## Analyzing the df (all_data.json)
this is just a generic break down of the data

df_death (DataFrame) :  adverse reactions that resulted in a death

### DIED column

In [85]:

break_down_columns(df,'DIED')
print('\n' + '-'*25 + '\n')




break down of DIED


,column,value,count,percent
0,DIED,nan,442738,98.62797
1,DIED,Y,6159,1.37203



-------------------------



### ER_VISIT column

In [86]:

print('did the adverse reaction result in an ER Visit')
break_down_columns(df,'ER_VISIT')
print('\n' + '-'*25 + '\n')


did the adverse reaction result in an ER Visit

break down of ER_VISIT


,column,value,count,percent
0,ER_VISIT,nan,448840,99.987302
1,ER_VISIT,Y,57,0.012698



-------------------------



### L_THREAT column

In [87]:

print('Life Threatening/Leathal Threat')
break_down_columns(df,'L_THREAT')
print('\n' + '-'*25 + '\n')



Life Threatening/Leathal Threat

break down of L_THREAT


,column,value,count,percent
0,L_THREAT,nan,441016,98.244363
1,L_THREAT,Y,7881,1.755637



-------------------------



### RECOVD column

In [88]:

print('did the patient recover?')
break_down_columns(df,'RECOVD')

print('\n' + '-'*25 + '\n')


did the patient recover?

break down of RECOVD


,column,value,count,percent
0,RECOVD,N,166634,37.120765
1,RECOVD,Y,160357,35.722449
2,RECOVD,U,77645,17.296841
3,RECOVD,nan,44261,9.859946



-------------------------



### the Age of the patient

In [89]:


print('the Age of the patient')
break_down_buckets(df,'AGE_YRS',[0,15,25,35,45,55,65,75,85,500])
break_down_buckets(df_death,'AGE_YRS',[0,15,25,35,45,55,65,75,85,500],message='***deaths only***')

print('\n' + '-'*25 + '\n')



the Age of the patient
  
column:  AGE_YRS 
buckets:  [0, 15, 25, 35, 45, 55, 65, 75, 85, 500]


,AGE_YRS,percent
bucket,,
"(0, 15]",7213,1.72
"(15, 25]",39882,9.49
"(25, 35]",65715,15.64
"(35, 45]",75234,17.90
"(45, 55]",73435,17.48
"(55, 65]",72923,17.35
"(65, 75]",55305,13.16
"(75, 85]",22755,5.42
"(85, 500]",7739,1.84


 ***deaths only*** 
column:  AGE_YRS 
buckets:  [0, 15, 25, 35, 45, 55, 65, 75, 85, 500]


,AGE_YRS,percent
bucket,,
"(0, 15]",9,0.16
"(15, 25]",50,0.87
"(25, 35]",101,1.76
"(35, 45]",183,3.19
"(45, 55]",325,5.67
"(55, 65]",822,14.34
"(65, 75]",1332,23.24
"(75, 85]",1531,26.71
"(85, 500]",1378,24.04



-------------------------



### the number of days between the vaccine and the adverse rection

In [90]:


print('the number of days between the vaccine and the adverse rection')
break_down_buckets(df,'NUMDAYS',[0,10,20,30,40,50,60])
break_down_buckets(df_death,'NUMDAYS',[0,10,20,30,40,50,60],message='***deaths only***')

print('\n' + '-'*25 + '\n')


the number of days between the vaccine and the adverse rection
  
column:  NUMDAYS 
buckets:  [0, 10, 20, 30, 40, 50, 60]


,NUMDAYS,percent
bucket,,
"(0, 10]",184478,82.06
"(10, 20]",20407,9.08
"(20, 30]",10518,4.68
"(30, 40]",4995,2.22
"(40, 50]",2552,1.14
"(50, 60]",1869,0.83


 ***deaths only*** 
column:  NUMDAYS 
buckets:  [0, 10, 20, 30, 40, 50, 60]


,NUMDAYS,percent
bucket,,
"(0, 10]",2524,56.66
"(10, 20]",784,17.60
"(20, 30]",497,11.16
"(30, 40]",302,6.78
"(40, 50]",195,4.38
"(50, 60]",153,3.43



-------------------------



### break down of the VAX_NAME column

In [138]:

print('break down of the VAX_NAME column')
df_VN = break_down_columns(df,'VAX_NAME')

print('\n***deaths only***')
df_DVN = break_down_columns(df_death,'VAX_NAME')

# df_CVN = df_VN.join(df_DVN[['value','count']],on='value')
df_DVN = df_DVN.rename(columns={"count": "death_count"})


print('calculating the death ratio ( death_count / count )')
df_CVN = pd.concat([df_VN,df_DVN[['death_count']]],join="inner",axis=1)
df_CVN['death_ratio'] = df_CVN['death_count'] / df_CVN['count']
df_CVN = df_CVN.drop(columns=['percent','death_count','count']) 
# df_CVN['death_percent'] = df_CVN['death_ratio']*100

display(df_CVN)

print('\n' + '-'*25 + '\n')

break down of the VAX_NAME column

break down of VAX_NAME


,column,value,count,percent
0,VAX_NAME,COVID19 (COVID19 (MODERNA)),197493,43.995170
1,VAX_NAME,COVID19 (COVID19 (PFIZER-BIONTECH)),205354,45.746352
2,VAX_NAME,COVID19 (COVID19 (UNKNOWN)),1024,0.228115
3,VAX_NAME,COVID19 (COVID19 (JANSSEN)),45026,10.030363



***deaths only***

break down of VAX_NAME


,column,value,count,percent
0,VAX_NAME,COVID19 (COVID19 (MODERNA)),2832,45.981491
1,VAX_NAME,COVID19 (COVID19 (PFIZER-BIONTECH)),2763,44.861179
2,VAX_NAME,COVID19 (COVID19 (UNKNOWN)),28,0.454619
3,VAX_NAME,COVID19 (COVID19 (JANSSEN)),536,8.702711


calculating the death ratio ( death_count / count )


,column,value,death_ratio
0,VAX_NAME,COVID19 (COVID19 (MODERNA)),0.014340
1,VAX_NAME,COVID19 (COVID19 (PFIZER-BIONTECH)),0.013455
2,VAX_NAME,COVID19 (COVID19 (UNKNOWN)),0.027344
3,VAX_NAME,COVID19 (COVID19 (JANSSEN)),0.011904



-------------------------



## Approximating actual adverse reaction numbers

VAERS only contains reported data and  
*'...fewer than 1% of vaccine adverse events are reported.'*  
Source: 
https://digital.ahrq.gov/sites/default/files/docs/publication/r18hs017045-lazarus-final-report-2011.pdf (page 6)


we will me multiplying the counts by 80 and 120,  
in order to get an approzimate min and max.  



In [91]:

def print_percent(vmin,vmax,label0,vcount,label1):
    """
    vmin: vaers min
    vmax: vaers max
    label0: vaers label
    vcount: vaxxed count
    label1: vaxxed label
    """
    print(
        '( {label0} / {lavel1} ) * 100\n'.format(
            label0=label0,
            lavel1=label1
            ),
        'min: ( {0:,} / {1:,} ) * 100 \n'.format(vmin,vcount),
        'max: ( {0:,} / {1:,} ) * 100 \n'.format(vmax,vcount),
        '{:.2f} %'.format((vmin/vcount)*100),
        '-',
        '{:.2f} %'.format((vmax/vcount)*100),
        '\n'
        )   

print_percent(
    len(df)*80,
    len(df)*120,
    'adverse reactions',
    vaxx,
    'vaxxed [1 or more shots]'
    )

print_percent(
    len(df_death)*80,
    len(df_death)*120,
    'adverse deaths',
    vaxx,
    'vaxxed [1 or more shots]'
    )

df_nrecovd = df[df['RECOVD']=='N']
print_percent(
    len(df_nrecovd)*80,
    len(df_nrecovd)*120,
    'no recovery',
    vaxx,
    'vaxxed [1 or more shots]'
    )

df_urecovd = df[df['RECOVD']=='U']
print_percent(
    (len(df_nrecovd) + (len(df_urecovd)*0.5))*80,
    (len(df_nrecovd) + (len(df_urecovd)*0.5))*120,
    'no recovery + (50% of unknowns)',
    vaxx,
    'vaxxed [1 or more shots]'
    )


( adverse reactions / vaxxed [1 or more shots] ) * 100
 min: ( 35,911,760 / 191,000,000 ) * 100 
 max: ( 53,867,640 / 191,000,000 ) * 100 
 18.80 % - 28.20 % 

( adverse deaths / vaxxed [1 or more shots] ) * 100
 min: ( 492,720 / 191,000,000 ) * 100 
 max: ( 739,080 / 191,000,000 ) * 100 
 0.26 % - 0.39 % 

( no recovery / vaxxed [1 or more shots] ) * 100
 min: ( 13,330,720 / 191,000,000 ) * 100 
 max: ( 19,996,080 / 191,000,000 ) * 100 
 6.98 % - 10.47 % 

( no recovery + (50% of unknowns) / vaxxed [1 or more shots] ) * 100
 min: ( 16,436,520.0 / 191,000,000 ) * 100 
 max: ( 24,654,780.0 / 191,000,000 ) * 100 
 8.61 % - 12.91 % 



## Women's Reproductive Symptoms

Why?
I have women in my life that were curious about this.

WRS_list (list) :  a list of symptoms that effect or could cause effects to a women's reproductive system
df_WRS (DataFrame) :  a dataframe that contains VAERS events that have at least 1 of the WRS symptoms

In [92]:
def symptom_filter_search(idf, search_list):
    """
    returns a dataframe pf vaers events where the patient 
    has had 1 or more of the symptoms on the list
    """
    print('this could take between 20sec-60sec')

    data = idf.to_dict(orient='records')

    search_list = [i.upper() for i in search_list]
    
    results = []
    for index,d in enumerate(data):
        # if index%5000 == 0:
        #     print('{:.2f}'.format(index/len(data)))
        try:
            d['SYMPTOMS'] = [i.upper() for i in d['SYMPTOMS'] if isinstance(i, str)]
        
            symptom_match = list(set(d['SYMPTOMS']) & set(search_list))
            d['SYMPTOMS_MATCH'] = symptom_match
            d['SYMPTOMS_MATCH_LENGTH'] = len(symptom_match)
            if len(symptom_match) > 0:
                results.append(d)
        except:
            pass

    return pd.DataFrame(results)

In [93]:
WRS_list = [
    'Intermenstrual bleeding',
    'Menopause',
    'Heavy menstrual bleeding',
    'dysmenorrhoea',
    'ABNORMAL UTERINE BLEEDING',
    'MATERNAL EXPOSURE BEFORE PREGNANCY',
    'MENSTRUATION IRREGULAR',
    'Oligomenorrhea',
    'OLIGOMENORRHOEA',
    'POLYMENORRHOEA',
    'MENSTRUAL DISORDER', 
    'OLIGOMENORRHOEA',
    'ANOVULATORY CYCLE',
    'OVULATION DELAYED',
    'BACTERIAL VAGINOSIS',
    'GYNAECOLOGICAL EXAMINATION ABNORMAL',
    'OVARIAN CYST',
    'BIOPSY UTERUS',
    'UTERINE LEIOMYOMA',
    'HOT FLUSH',
    'BREAST TENDERNESS',
    'BREAST SWELLING',
    'BREAST PAIN',
    'VAGINAL HAEMORRHAGE'
    ]
WRS_list = [i.upper() for i in WRS_list]

In [113]:
df_WRS = symptom_filter_search(df,WRS_list)

print('df_WRS.head(5)')
display(df_WRS.head(5))


this could take between 20sec-60sec
df_WRS.head(5)


,VAERS_ID,RECVDATE,STATE,AGE_YRS,CAGE_YR,CAGE_MO,SEX,RPT_DATE,SYMPTOM_TEXT,DIED,DATEDIED,L_THREAT,ER_VISIT,HOSPITAL,HOSPDAYS,X_STAY,DISABLE,RECOVD,VAX_DATE,ONSET_DATE,NUMDAYS,LAB_DATA,V_ADMINBY,V_FUNDBY,OTHER_MEDS,CUR_ILL,HISTORY,PRIOR_VAX,SPLTTYPE,FORM_VERS,TODAYS_DATE,BIRTH_DEFECT,OFC_VISIT,ER_ED_VISIT,ALLERGIES,VAX_TYPE,VAX_MANU,VAX_LOT,VAX_DOSE_SERIES,VAX_ROUTE,VAX_SITE,VAX_NAME,SYMPTOMS,COVID_VAX,SYMPTOMS_MATCH,SYMPTOMS_MATCH_LENGTH
0,902796,12/16/2020,TX,34.0,34.0,NaN,F,NaN,I inserted my NuvaRing birth control on 12/14/...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,12/15/2020,12/16/2020,1.0,NaN,PVT,NaN,Methylphenidate Citalopram Buspirone Tylenol,None,I normally get a migraine a day or two after I...,NaN,NaN,2,12/16/2020,NaN,NaN,NaN,None,COVID19,PFIZER\BIONTECH,EH9899,1,SYR,LA,COVID19 (COVID19 (PFIZER-BIONTECH)),"[CONDITION AGGRAVATED, FEELING ABNORMAL, HOT F...",1,[HOT FLUSH],1
1,903202,12/17/2020,NE,41.0,41.0,NaN,F,NaN,Severe dizzy spell about 5-10 following inject...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,12/16/2020,12/16/2020,0.0,None,OTH,NaN,"Buspar, propranolol, escitalopram, crestor, a...",None,"Migraine, anxiety, depression, hyperlipidemia",NaN,NaN,2,12/17/2020,NaN,NaN,NaN,Aimovig,COVID19,PFIZER\BIONTECH,EH9899,1,IM,RA,COVID19 (COVID19 (PFIZER-BIONTECH)),"[DIZZINESS, HOT FLUSH, VISUAL IMPAIRMENT]",1,[HOT FLUSH],1
2,903247,12/17/2020,AR,42.0,42.0,NaN,F,NaN,I had arm pain. Yesterday I had a hot flash wi...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,12/15/2020,12/16/2020,1.0,No,UNK,NaN,"I take Estradiol, Metprolol, Lexapro, Multivit...",No,I have SVT,NaN,vsafe,2,12/17/2020,NaN,NaN,NaN,I am allergic to Sulphur,COVID19,PFIZER\BIONTECH,NaN,UNK,IM,LA,COVID19 (COVID19 (PFIZER-BIONTECH)),"[DIARRHOEA, HOT FLUSH, PAIN IN EXTREMITY, VOMI...",1,[HOT FLUSH],1
3,903329,12/18/2020,PA,46.0,46.0,NaN,F,NaN,"hot flushing feeling, light headed, legs heavy...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,12/17/2020,12/17/2020,0.0,NaN,PVT,NaN,NaN,NaN,NaN,NaN,NaN,2,12/18/2020,NaN,NaN,NaN,NaN,COVID19,PFIZER\BIONTECH,EK5730,1,IM,RA,COVID19 (COVID19 (PFIZER-BIONTECH)),"[DIZZINESS, HOT FLUSH, LIMB DISCOMFORT]",1,[HOT FLUSH],1
4,903345,12/18/2020,VA,37.0,37.0,NaN,F,NaN,"Right arm soreness, headache, hot flashes.",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,12/16/2020,12/17/2020,1.0,NaN,WRK,NaN,NaN,None,None,NaN,NaN,2,12/18/2020,NaN,NaN,NaN,None,COVID19,PFIZER\BIONTECH,NaN,UNK,NaN,RA,COVID19 (COVID19 (PFIZER-BIONTECH)),"[HEADACHE, HOT FLUSH, PAIN IN EXTREMITY]",1,[HOT FLUSH],1


### What percent of women experienced WRS during their adverse reactions?

(women experiencing reproductive symptoms ➗ number of women in VAERS data) * 100

WRS_ratio (float) : count WRS / count VAERS

In [108]:
#get count of women in VAERS data 

w_df = df[df['SEX']=='F']
u_df = df[df['SEX']=='U']
w_count = len(w_df) + (len(u_df)/2) #half of unknown

WRS_ratio = (len(df_WRS)/w_count)

print( '{:.2f} %'.format(WRS_ratio*100))

file_name = os.path.join(DATAPATH,'WRS.csv')
df_WRS.to_csv(file_name)

print('VAERS records of women experiencing reproductive symptoms have been saved.')
print('saved: ',file_name)



3.10 %
VAERS records of women experiencing reproductive symptoms have been saved.
saved:  .\AllVAERSDataCSVS\WRS.csv


### Approximate the number of WRS in reality

since '~63% of the people who are vaccinated are women'
source: https://www.statista.com/statistics/1212103/share-of-persons-initiating-covid-vaccinations-by-gender-us-first-month/

we will be multiplying the total number of people vaccinated (1 or more shots) by 0.63 to get the count of women vaccinated.

women_vaxx (float) : an approximate number of women who have had 1 or more vaccine shot.

>note: 0.63 is an estimate, and the actual could be somewhere between 0.50 and 0.70

In [109]:
women_vaxx = vaxx * 0.63


### approximate real WRS
if we assume that the VAERS data is a random sample (or close to it)  
then the ratio of WRS systems should be the same...

and thus we can get an approximate number of women that would be experiencing reproductive symptoms by multiplying the number of vaxxed women by the ratio

WRS (float) : the approximate number of actual women experiencing reproductive symptoms
min_WRS (float) : WRS * 0.80
max_WRS (float) : WRS * 1.20

In [110]:
WRS = women_vaxx * WRS_ratio
min_WRS = WRS*0.80
max_WRS = WRS*1.20

In [111]:
cl = [35,5,15]
print_row(['total vaxxed (1 or more)','---','{:,.2f}'.format(vaxx)],column_lengths=cl)
print_row(['women vaxxed ~0.63%','---','{:,.2f}'.format(women_vaxx)],column_lengths=cl)
print_row(['repro sympt / women count','---','{:,.4f}'.format(WRS_ratio)],column_lengths=cl)
print_row(['(repro sympt / women count) * 100','---','{:,.2f} %'.format(WRS_ratio*100)],column_lengths=cl)
print_row(['women w/ repro symptoms','---','{:,.2f}'.format(WRS)],column_lengths=cl)
print_row(['min women w/ repro symptoms','---','{:,.2f}'.format(min_WRS)],column_lengths=cl)
print_row(['min women w/ repro symptoms','---','{:,.2f}'.format(max_WRS)],column_lengths=cl)


total vaxxed (1 or more)           ---  191,000,000.00 
women vaxxed ~0.63%                ---  120,330,000.00 
repro sympt / women count          ---  0.0310         
(repro sympt / women count) * 100  ---  3.10 %         
women w/ repro symptoms            ---  3,731,066.18   
min women w/ repro symptoms        ---  2,984,852.94   
min women w/ repro symptoms        ---  4,477,279.41   


### Where do the WRS (Women reproductive symptoms) rank against with the other symptoms?

In [112]:
temp = []

for index,i in enumerate(all_symptoms):
    if i[0].upper() in WRS_list:
        temp.append(
            {
                'index':index,
                'symptoms':i[0],
                'count': '{:,.2f}'.format(i[1]),
                'percent': '{:.2f}'.format((i[1]/len(all_symptoms))*100) 
            }
        )

temp = pd.DataFrame(temp)
display(temp)

,index,symptoms,count,percent
0,126,HOT FLUSH,"2,715.00",29.87
1,145,HEAVY MENSTRUAL BLEEDING,"2,234.00",24.58
2,167,MENSTRUATION IRREGULAR,"1,683.00",18.51
3,196,MENSTRUAL DISORDER,"1,406.00",15.47
4,230,BREAST PAIN,"1,046.00",11.51
5,247,DYSMENORRHOEA,958.00,10.54
6,272,VAGINAL HAEMORRHAGE,834.00,9.17
7,305,INTERMENSTRUAL BLEEDING,694.00,7.63
8,460,BREAST SWELLING,371.00,4.08
9,489,OLIGOMENORRHOEA,343.00,3.77
